In [ ]:
from lib.utils import load_eeg_label_pair,test_evaluation,cm_grid
from lib.env import *
from tqdm import tqdm
from lib.models import MLP
import torch
from torch.utils.data import TensorDataset,DataLoader
import matplotlib.pyplot as plt
from lib.datasets import Dataset2p0
from lib.utils import get_cross_validation_split_for_fold,load_eeg_label_pair,evaluate

DATASET = 'w1_cv_0'
trainloader = DataLoader(Dataset2p0(dir=f'{DATASET}/train/',labels=f'{DATASET}/y_train.pt'),batch_size=64,shuffle=True)
devloader = DataLoader(Dataset2p0(dir=f'{DATASET}/dev/',labels=f'{DATASET}/y_dev.pt'),batch_size=64,shuffle=True)

In [ ]:
model = MLP().to(DEVICE)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
lossi = []

model.train()

for epoch in tqdm(range(10)):
    loss_total = 0
    for (Xi,yi) in trainloader:
        Xi,yi = Xi.to(DEVICE),yi.to(DEVICE)
        logits = model(Xi)
        loss = criterion(logits,yi)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_total += loss.item()
    lossi.append(loss_total/len(trainloader))

plt.plot(lossi)
print(torch.tensor(lossi[-len(trainloader):]).mean())

In [ ]:
train_ids,test_ids = get_cross_validation_split_for_fold(foldi=0)
Xt,yt = load_eeg_label_pair(test_ids[0])
dataloader = DataLoader(TensorDataset(Xt,yt),batch_size=256,shuffle=False)
loss,y_true,y_pred,y_logits = evaluate(dataloader,model,criterion)

In [ ]:
cm_grid(y_true=y_true,y_pred=y_pred)

In [ ]:
import matplotlib.pyplot as plt
plt.stackplot(torch.linspace(0,1,100),y_logits[:100,0],y_logits[:100,1],y_logits[:100,2])
import plotly.express as px
fig = px.area(y_logits)
fig.show(renderer='browser')